In [1]:
import numpy as np
import matplotlib.pyplot as plt
import vtk
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
import sys
sys.path.append('/home/thayer/Desktop/DavidCS/ubuntu_partition/code/pydar/')
import pydar

%load_ext autoreload
%autoreload 2

pdal not imported, functions relying on it will fail
opencv was not imported, functions relying on it will fail


In [2]:
project_path = '/media/thayer/Data/mosaic_lidar/ROV/'
project_name = 'mosaic_rov_220220.RiSCAN.RiSCAN'

suffix='slfsnow'

project = pydar.Project(project_path, project_name, import_mode='read_scan',
                        class_list='all', las_fieldnames=['Points',
                                                            'Classification',
                                                            'PointId'],
                        suffix=suffix)

In [3]:
# Apply the sop transform and create pointwise z uncertainty
project.apply_transforms(['sop'])
project.create_z_sigma()

In [4]:
# Now create normalized heights
x0 = 40.05
y0 = 65.05
nx = 200
ny = 200
dx = 0.1
dy = 0.1

bounds = (x0-5, x0+nx*dx+5, y0-5, y0+ny*dy+5, -10, 0)
project.create_empirical_cdf(bounds)
project.create_normalized_heights()

/home/thayer/Desktop/DavidCS/ubuntu_partition/code/pydar/pydar.py:1856: UserWarning: You are passing a SingleScan's PolyData withoutit's history.
  warnings.warn("You are passing a SingleScan's PolyData without" +


In [5]:
# Display normalized heights

project.display_project(-3, 3, field='norm_height')

In [8]:
# Examine the polydata we'll be using to create image
pdata = project.get_merged_points(x0=x0, y0=y0, wx=nx*dx, wy=ny*dy)
print(pdata)

vtkPolyData (0x56316a1e76f0)
  Debug: Off
  Modified Time: 157068564
  Reference Count: 1
  Registered Events: (none)
  Information: 0x563150435cb0
  Data Released: False
  Global Release Data: Off
  UpdateTime: 157068570
  Field Data:
    Debug: Off
    Modified Time: 157068329
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 14380
  Number Of Cells: 7
  Cell Data:
    Debug: Off
    Modified Time: 157068510
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0x56316a1e7cf0)
          Event: 33
          EventName: ModifiedEvent
          Command: 0x56316a1e79b0
          Priority: 0
          Tag: 1
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
    Copy Tuple Flags: ( 1 1 1 1 1 1 1 1 1 1 1 )
    Interpolate Flags: ( 1 1 1 1 1 0 0 1 1 1 1 )
    Pass Through Flags: ( 1 1 1 1 1 1 1 1 1 1 1 )
    Scalars: (none)
    

In [9]:
norm_height = vtk_to_numpy(pdata.GetPointData().GetArray('norm_height'))
print(norm_height.min())
print(norm_height.max())

norm_z_sigma = vtk_to_numpy(pdata.GetPointData().GetArray('norm_z_sigma'))
print(norm_z_sigma.min())
print(norm_z_sigma.max())
print((norm_z_sigma<0).sum())

-4.069133
3.2532096
-3.441732
0.5273427
7


In [11]:
# Test our function for using gpytorch to create an image from points

yaw = 0

leafsize = 500000
lengthscale = 10
outputscale = 0.15

project.merged_points_to_image(nx, ny, dx, dy, x0, y0, leafsize, lengthscale, outputscale)

In [12]:
z_min = -3.5
z_max = -2

project.display_image(z_min, z_max)

In [21]:
z_min = -3.5
z_max = -2

project.display_image(z_min, z_max, warp_scalars=True)

In [31]:
v_min = 0
v_max = 0.1

project.display_image(v_min, v_max, field='Elevation',warp_scalars=True, color_field='z_ci')

pdal not imported, functions relying on it will fail
opencv was not imported, functions relying on it will fail


In [14]:
# Repeat, now with corner_coords
corner_coords = np.array([[40, 65, 0],
                         [40, 85, 0],
                         [60, 85, 0]])
leafsize = 500000
lengthscale = 10
outputscale = 0.15

project.merged_points_to_image(nx, ny, dx, dy, x0, y0, leafsize, lengthscale, outputscale,
                              corner_coords=corner_coords)

pdal not imported, functions relying on it will fail
opencv was not imported, functions relying on it will fail


/home/thayer/Desktop/DavidCS/ubuntu_partition/code/pydar/pydar.py:4983: UserWarning: Still has temporary fix for negative norm z sigma
  warnings.warn('Still has temporary fix for negative norm z sigma')


In [15]:
z_min = -3.5
z_max = -2

project.display_image(z_min, z_max)

In [17]:
# repeat with kdtree

leafsize = 5000
lengthscale = 10
outputscale = 0.15

project.merged_points_to_image(nx, ny, dx, dy, x0, y0, leafsize, lengthscale, outputscale)

pdal not imported, functions relying on it will fail
opencv was not imported, functions relying on it will fail


/home/thayer/Desktop/DavidCS/ubuntu_partition/code/pydar/pydar.py:4983: UserWarning: Still has temporary fix for negative norm z sigma
  warnings.warn('Still has temporary fix for negative norm z sigma')


In [18]:
z_min = -3.5
z_max = -2

project.display_image(z_min, z_max)

In [19]:
# Repeat, now with corner_coords and kdtree
corner_coords = np.array([[40, 65, 0],
                         [40, 85, 0],
                         [60, 85, 0]])
leafsize = 5000
lengthscale = 10
outputscale = 0.15

project.merged_points_to_image(nx, ny, dx, dy, x0, y0, leafsize, lengthscale, outputscale,
                              corner_coords=corner_coords)

In [20]:
z_min = -3.5
z_max = -2

project.display_image(z_min, z_max)

In [5]:
# Try saving and then reading from file
project.write_image(suffix=suffix, name='test')

In [6]:
# Delete the project
del project

In [4]:
# Create a new project and read the image
project_path = '/media/thayer/Data/mosaic_lidar/ROV/'
project_name = 'mosaic_rov_220220.RiSCAN.RiSCAN'

suffix='slfsnow'

project = pydar.Project(project_path, project_name, import_mode='empty',
                        suffix=suffix)

In [5]:
project.read_image(suffix=suffix, name='test')

In [6]:
z_min = -3.5
z_max = -2

project.display_image(z_min, z_max, warp_scalars=True)